## Experimental Workflows


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, alignment, patterning
from fibsem.structures import ImageSettings, BeamType
from fibsem.ui.utils import _draw_milling_stages_on_image


import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint
import logging

microscope, settings = utils.setup_session()





### Automated Quality Control

In [ ]:
# ref: https://pure.mpg.de/rest/items/item_2415133/component/file_3148891/content
# ref: https://www.biorxiv.org/content/10.1101/2024.02.21.581285v1.full.pdf
# target acquisition
# set up imaging (voltage = 3kv, line integration= 20, dwell time 100ns, res [3072x2188])

# constrast test
# set electron imaging (voltage = 3kv, line integration = 1, dwell time 200ns)
# set up top milling pattern (rectangule) (stepsize = 100nm)

# steps:
# do standard rough milling
# loop until target is found

# do contrast test polishing

protocol = {
    "polish": {
        "stages": [{
    "application_file": "autolamella",
    "cross_section": "CleaningCrossSection",
    "hfw": 80e-6,
    "height": 6.0e-07,
    "width": 6.0e-06,
    "depth": 4.0e-07,
    "milling_current": 6.0e-11,
    "milling_voltage": 3.0e3,
    "type": "Rectangle",
        }
    ],
    "point": {
        "x": 0.0,
        "y": 0.0,}
    }
}
stages = patterning.get_milling_stages("polish", protocol)

# beam settings
initial_voltage = microscope.get("voltage", BeamType.ELECTRON)
microscope.set("voltage", 3000, BeamType.ELECTRON)

# imaging settings
settings.image.hfw = 40e-6
settings.image.save = True

# contrast test settings
threshold = 100
step_size = 100e-9
step_limit = 10
n_step = 0
while True:

    # draw rectangle
    # _draw_milling_stages_on_image(image, stages)    

    # mill pattern
    # milling.mill_stages(microscope, stages)

    # acquire image
    settings.image.filename = f"thickness-dependent-contrast-test-{n_step}"
    image = acquire.acquire_image(microscope, settings.image)

    # measure brightness and contrast
    brightness = np.mean(image.data)
    contrast = np.std(image.data)
    logging.info({"msg": "thickness-dependent-contrast-test",  
                  "brightness": brightness, "contrast": contrast, "threshold": threshold, 
                  "n_step": n_step, "step_size": step_size, "step_limit": step_limit})

    if brightness < threshold:
        break

    # manual validation
    # response = input(f"Has the contrast faded? (y/n)")

    # if response == 'y':
        # break

    # ml classifier
    # TODO: implement a classifier to determine if the contrast has faded

    n_step += 1
    if n_step > step_limit:
        break

    # move pattern down by step
    for stage in stages:
        stage.pattern.point.y -= step_size


# restore beam settings
microscope.set("voltage", initial_voltage, BeamType.ELECTRON)

